<a href="https://colab.research.google.com/github/enzocfb/general/blob/main/EDA_2023_deductivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
"""
Analisis de Deductivos para analizar causas
Impacto :

input:
      Consolidato total de Evaluados por UGS(Formato 06) para todos los años
      Deductivos reportados por UGT
      Fechas referenciales por grupos

output:
      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"

"""

'\nAnalisis de Deductivos para analizar causas\nImpacto :\n\ninput:\n      Consolidato total de Evaluados por UGS(Formato 06) para todos los años\n      Deductivos reportados por UGT\n      Fechas referenciales por grupos\n\noutput:\n      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"\n\n'

In [43]:
import pandas as pd
import numpy as np

In [44]:
file = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Formato06Completo.xlsx'
file_deductivos= '/content/drive/MyDrive/@@@Dev/mvcs/Data/LISTADO DE DEDUCTIVOS A (UGS).xlsx'
file_input_nucleo = '/content/drive/MyDrive/@@@Dev/mvcs/Data/NUCLEO - REPORTE COMPLEMENTARIO.xlsx'

file_output_1 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs no encontrados.xlsx'
file_output_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs si encontrados.xlsx'
file_output_analisis = '/content/drive/MyDrive/@@@Dev/mvcs/Data/deductivos-completo-analysis.xlsx'
file_output_3 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp.xlsx'



In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str})
df.head(3)

# CRACION DE ESTADOS PARA TABLA INICIAL



,NUMERO,ORIGEN,PORTAFOLIO,PORTAFOLIO_T,GRUPO,NRO ENCUESTA,CUV,CODIGO_CCPP,UBIGEO_temp,UBIGEO,...,CRITERIO_SOCIAL,CRITERIO_TECNICO,SOCIAL ESTADO DE LA FICHA,TÉCNICA,OBSERVACIONES,FECHA CADUCIDAD CSE,FECHA CONSULTA CSE,ACEPTA_ENTREVISTA,ESTADO_EVALUADO_T,FEC_EVALUACION
0,1,PP068,02-2024,2024-02,2,1,0808050259454185562402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,REGISTRADO,NaN,2028-08-19 00:00:00,2023-09-21 02:30:56,SI,SI,14-09-2023
1,2,PP068,02-2024,2024-02,2,2,0808050259000000002402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,SIN REGISTRO,NaN,NaN,NaN,Familia Ausente,NO,14-09-2023
2,3,PP068,02-2024,2024-02,2,3,0808050259475809342402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,REGISTRADO,NaN,2024-10-09 00:00:00,2023-09-21 02:37:17,SI,SI,14-09-2023


In [51]:
# Seleccionar campos requeridos para el analisis y construir las features

columns_selected_df = ['NUMERO', 'ORIGEN', 'PORTAFOLIO_T', 'CUV', 'CODIGO_CCPP', 'UBIGEO', 'DEPARTAMENTO','PROVINCIA', 'DISTRITO', 'CENTRO_POBLADO', 'COMUNIDAD', 'PATERNO_JEFE_HOGAR', 'NOMBRES_JEFE_HOGAR', 'DNI_JEFE_HOGAR', 'SEXO_JEFE_HOGAR', 'EDAD_JEFE_HOGAR', 'FEC_NAC_JEFE_HOGAR', 'CANT', 'CTD_DISCAPACIDAD', 'ALTURA', 'LATITUD', 'LONGITUD', 'CLASIFICACION_SOCIOECONOMICA_CSE', 'VIABILIDAD_TOTAL', 'VIABILIDAD_SOCIAL', 'VIABILIDAD_TECNICA', 'ACEPTA_ENTREVISTA', 'FEC_EVALUACION', 'CRITERIO_SOCIAL', 'CRITERIO_TECNICO']
df = df[columns_selected_df]

In [52]:
print(df.info())
print(df.shape[0])

# -- 26450, esta bien, coincide con el Excel


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26450 entries, 0 to 26449
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NUMERO                            26450 non-null  int64  
 1   ORIGEN                            26450 non-null  object 
 2   PORTAFOLIO_T                      26450 non-null  object 
 3   CUV                               26450 non-null  object 
 4   CODIGO_CCPP                       26450 non-null  object 
 5   UBIGEO                            26450 non-null  object 
 6   DEPARTAMENTO                      26450 non-null  object 
 7   PROVINCIA                         26450 non-null  object 
 8   DISTRITO                          26450 non-null  object 
 9   CENTRO_POBLADO                    26450 non-null  object 
 10  COMUNIDAD                         15300 non-null  object 
 11  PATERNO_JEFE_HOGAR                25454 non-null  object 
 12  NOMB

In [53]:
import numpy as np
# Creamos ESTADOS : REGISTRADO, EVALUACION,  VIABILIDAD,
constante_estado_registrado = 'REGISTRADO'

constante_estado_evaluacion_evaluado = "EVALUADO"
constante_estado_evaluacion_rechaza = "RECHAZA_EVALUACION"
constante_estado_evaluacion_otros = 'OTROS_NO_EVALUADO'

constante_estado_viabilidad_viable = "VIABLE"
constante_estado_viabilidad_no_viable = "NO VIABLE"
constante_estado_sininfo = "SIN INFORMACION"


constante_estado_deductivo = "DEDUCTIVO"
constante_estado_no_deductivo = "NO_DEDUCTIVO"

df['ESTADIO_REGISTRADO'] = constante_estado_registrado
df['ESTADO_EVALUACION'] = df['ACEPTA_ENTREVISTA'].apply(lambda x: 'SI' if x == "SI" else 'NO')

df['ESTADO_EVALUACION'] = np.where(df['ACEPTA_ENTREVISTA'] == 'SI', constante_estado_evaluacion_evaluado,
                        np.where(df['ACEPTA_ENTREVISTA'] == 'Rechazo de evaluación / Intervención', constante_estado_evaluacion_rechaza, constante_estado_evaluacion_otros))


# df['ESTADO_VIABLE'].fillna(constante_estado_viabilidad_sininfo, inplace = True)

df['ESTADO_VIABLE'] = np.where(df['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable, constante_estado_viabilidad_viable,
                        np.where(df['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_no_viable, constante_estado_viabilidad_no_viable, constante_estado_sininfo))



In [54]:
df.head(3)

,NUMERO,ORIGEN,PORTAFOLIO_T,CUV,CODIGO_CCPP,UBIGEO,DEPARTAMENTO,PROVINCIA,DISTRITO,CENTRO_POBLADO,...,VIABILIDAD_TOTAL,VIABILIDAD_SOCIAL,VIABILIDAD_TECNICA,ACEPTA_ENTREVISTA,FEC_EVALUACION,CRITERIO_SOCIAL,CRITERIO_TECNICO,ESTADIO_REGISTRADO,ESTADO_EVALUACION,ESTADO_VIABLE
0,1,PP068,2024-02,0808050259454185562402,0808050259,080805,CUSCO,ESPINAR,PALLPATA,LLAIQUEPAMPA,...,VIABLE,SOCIALMENTE VIABLE,VIABLE,SI,14-09-2023,NaN,NaN,REGISTRADO,EVALUADO,VIABLE
1,2,PP068,2024-02,0808050259000000002402,0808050259,080805,CUSCO,ESPINAR,PALLPATA,LLAIQUEPAMPA,...,NaN,NaN,NaN,Familia Ausente,14-09-2023,NaN,NaN,REGISTRADO,OTROS_NO_EVALUADO,SIN INFORMACION
2,3,PP068,2024-02,0808050259475809342402,0808050259,080805,CUSCO,ESPINAR,PALLPATA,LLAIQUEPAMPA,...,VIABLE,SOCIALMENTE VIABLE,VIABLE,SI,14-09-2023,NaN,NaN,REGISTRADO,EVALUADO,VIABLE


In [55]:
df.to_excel(file_output_3)

In [56]:
# Assuming df is your DataFrame
# Create a sample DataFrame for illustration purposes
data = {'acepta': ['SI', 'NO', 'SI', 'NO', 'SI'],
        'other_column': [10, 20, 30, 40, 50]}
df22 = pd.DataFrame(data)
df22

# Create the "ESTADO" column based on the "acepta" column
df22['ESTADO'] = df22['acepta'].apply(lambda x: 'SI' if x == 'SI' else 'NO')

# Display the DataFrame
print(df22)


  acepta  other_column ESTADO
0     SI            10     SI
1     NO            20     NO
2     SI            30     SI
3     NO            40     NO
4     SI            50     SI


In [57]:
df_deductivos = pd.read_excel(file_deductivos)
df_nucleo = pd.read_excel(file_input_nucleo)

In [58]:
df_nucleo.head(3)

,CONVENIO,COD UGT,FEC_APROB_EXP_TECNICO,FEC_ACTA_INICIO_CONSTRUCCION,GRUPO
0,013-2017-LOR/VMVU/PNVR,013-2017,2017-03-13,2023-06-19,GRUPO 4B
1,006-2022-PUN/VMVU/PNVR,006-2022,2022-03-25,2023-07-17,GRUPO 2
2,007-2022-PUN/VMVU/PNVR,007-2022,2022-03-25,2023-06-19,GRUPO 2


In [59]:
df_deductivos_nucleo = pd.merge(df_deductivos, df_nucleo,  on = 'CONVENIO', how = 'left')

# ESTADO : CREAMOS EL ESTADO DEDUCTIVO PARA ESTA TABLA
df_deductivos_nucleo['ESTADO_DEDUCTIVO'] = 'DEDUCTIVO'


In [60]:
print(df_deductivos.shape[0])
print(df_deductivos_nucleo.shape[0])
# print(df_deductivos_nucleo.head())

df_deductivos_nucleo.to_excel(file_output_3)
# print(df_deductivos_nucleo.info())

1017
1017


In [61]:
columns_selected_deductivos = ['ID', 'CUV','DNI','CODIGO_UGT','CONVENIO','NOMBRE','AP_PATERNO','DEDUCTIVO','CARTA','MOTIVO','HT','FEC_APROB_EXP_TECNICO','FEC_ACTA_INICIO_CONSTRUCCION','ESTADO_DEDUCTIVO']

df_deductivos_nucleo = df_deductivos_nucleo[columns_selected_deductivos]

df_join = pd.merge(df, df_deductivos_nucleo, on = 'CUV')
df_join_left = pd.merge(df, df_deductivos_nucleo, on = 'CUV', how = 'left', suffixes=('_F06', '_deductivo'))

# 130 registros

In [62]:
df_join_left.to_excel(file_output_analisis)

In [63]:
##########################################################################3
# CALCULOS POST PROCESO
##########################################################################3

# Completar ESTADO DEDUCTIVO , luego del JOIN

df_temp = df_join_left

condition_1 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & (df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )
condition_2 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & ~(df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )

df_temp.loc[condition_1, 'ESTADO_DEDUCTIVO'] = constante_estado_no_deductivo
df_temp.loc[condition_2, 'ESTADO_DEDUCTIVO'] = constante_estado_sininfo
df_temp['ESTADO_DEDUCTIVO'].fillna(constante_estado_deductivo, inplace=True)



In [64]:
df_join_left = df_temp

In [65]:
print(df.shape[0])
print(df_join.shape[0])
print(df_join_left.shape[0])
# df_join.head()
df_join_left.to_excel(file_output_analisis)

26450
130
26450


In [66]:
df1 = df_deductivos
df2 = df

rows_not_in_df = df1[~df1['CUV'].isin(df2['CUV'])]
rows_in_df = df1[df1['CUV'].isin(df2['CUV'])]


In [67]:
rows_not_in_df06.to_excel(file_output_1)
rows_in_df06.to_excel(file_output_2)

NameError: ignored

In [ ]:
rows_not_in_df.shape[0]

In [ ]:
import pandas as pd

# Sample DataFrames (Replace these with your actual DataFrames)
data1 = {'key': [1, 2, 3, 4, 5],
         'value': ['A', 'B', 'C', 'D', 'E']}
data2 = {'key': [2, 4, 5],
         'value': ['B', 'D', 'E']}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
print(df1)
print(df2)

# Select rows in df1 that are not in df2 based on 'key'
rows_not_in_df2 = df1[~df1['key'].isin(df2['key'])]

print(rows_not_in_df2)
